In [13]:
import pulp
import json
import math

def load_and_preprocess_data(filename="dataset.json"):
    """
    Loads data and pre-calculates feasible connections and flight times.
    This function *only* reads from the file.
    """
    try:
        with open(filename) as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Error: {filename} not found.")
        return None
    
    # Data Extraction
    flights_data = data['flights']
    tails_data = data['tails']
    costs = data['costs']

    # Data Structuring
    flight_ids = [f['flight_id'] for f in flights_data]
    tail_ids = [t['tail_id'] for t in tails_data]
    
    flight_times = {f['flight_id']: f['end_time'] - f['start_time'] for f in flights_data}
    flights_map = {f['flight_id']: f for f in flights_data}
    tails_map = {t['tail_id']: t for t in tails_data}

    # Pre-calculate Feasible Connections 
    # This is critical for the delay objective.
    # feasible_connections[(t, i, j)] = buffer_time
    feasible_connections = {}
    
    for t_id in tail_ids:
        min_turn = tails_map[t_id]['min_turnaround']
        for f_i in flights_data:
            for f_j in flights_data:
                if f_i['flight_id'] == f_j['flight_id']:
                    continue
                
                # 1. Check location
                if f_i['dest'] == f_j['origin']:
                    # 2. Check time
                    gap = f_j['start_time'] - f_i['end_time']
                    if gap >= min_turn:
                        buffer = gap - min_turn
                        feasible_connections[(t_id, f_i['flight_id'], f_j['flight_id'])] = buffer
    
    return {
        "flight_ids": flight_ids,
        "tail_ids": tail_ids,
        "costs": costs,
        "flight_times": flight_times,
        "feasible_connections": feasible_connections
    }

def solve_tail_assignment_mip(data, scenario_name, util_imbalance_limit=None, min_buffer_required=None):
    """
    Solves the full 3-objective MIP problem using the Epsilon-Constraint method.
    
    Main Objective: Minimize Cost
    Constraint 1: Workload imbalance <= util_imbalance_limit
    Constraint 2: Total buffer time >= min_buffer_required
    """
    
    # 1. Unpack Data 
    flight_ids = data['flight_ids']
    tail_ids = data['tail_ids']
    costs = data['costs']
    flight_times = data['flight_times']
    feasible_connections = data['feasible_connections']

    # 2. Model Initialization 
    prob = pulp.LpProblem(f"Tail_Assignment_{scenario_name}", pulp.LpMinimize)

    # 3. Decision Variables

    # x[t][f] = 1 if tail 't' is assigned to flight 'f'
    x = pulp.LpVariable.dicts("x", (tail_ids, flight_ids), 0, 1, pulp.LpBinary)

    # y[t][i][j] = 1 if tail 't' flies flight 'i' *then* flight 'j'
    y = pulp.LpVariable.dicts("y", feasible_connections.keys(), 0, 1, pulp.LpBinary)

    # Helper variables for utilization
    w_max = pulp.LpVariable("Workload_Max", 0)
    w_min = pulp.LpVariable("Workload_Min", 0)

    # 4. Define Objectives (as expressions) 
    
    # Obj 1: Total Cost
    obj_cost = pulp.lpSum(
        costs[t][f] * x[t][f] 
        for t in tail_ids 
        for f in flight_ids
    )

    # Obj 2: Workload Imbalance
    workload = {
        t: pulp.lpSum(flight_times[f] * x[t][f] for f in flight_ids) 
        for t in tail_ids
    }
    obj_utilization_imbalance = w_max - w_min

    # Obj 3: Total Buffer Time
    obj_total_buffer = pulp.lpSum(
        buffer * y[(t, i, j)]
        for (t, i, j), buffer in feasible_connections.items()
    )

    # 5. Set Main Objective
    prob += obj_cost, "Total_Cost_To_Minimize"

    # 6. Constraints

    # C1: Each flight is assigned exactly once
    for f in flight_ids:
        prob += pulp.lpSum(x[t][f] for t in tail_ids) == 1, f"C1_Assign_{f}"

    # C2: Flow Conservation (Links x and y)
    # If flight 'i' is assigned to tail 't' (x[t,i] = 1),
    # it must have at most one preceding flight and at most one subsequent flight.
    for t in tail_ids:
        for f_i in flight_ids:
            flights_into_i = pulp.lpSum(
                y[(t, f_j, f_i)] 
                for (t_key, f_j, f_i_key), _ in feasible_connections.items() 
                if t_key == t and f_i_key == f_i
            )
            flights_out_of_i = pulp.lpSum(
                y[(t, f_i, f_j)]
                for (t_key, f_i_key, f_j), _ in feasible_connections.items()
                if t_key == t and f_i_key == f_i
            )
            
            prob += flights_into_i <= x[t][f_i], f"C2_In_{t}_{f_i}"
            prob += flights_out_of_i <= x[t][f_i], f"C3_Out_{t}_{f_i}"

    # C3: Utilization helper constraints
    for t in tail_ids:
        prob += w_max >= workload[t], f"C4_W_Max_{t}"
        prob += w_min <= workload[t], f"C4_W_Min_{t}"

    # C4: Epsilon-Constraints (This is how we trade off)
    if util_imbalance_limit is not None:
        prob += obj_utilization_imbalance <= util_imbalance_limit, "C5_Util_Limit"
    
    if min_buffer_required is not None:
        prob += obj_total_buffer >= min_buffer_required, "C6_Buffer_Limit"

    # 7. Solve 
    print(f"\n Solving for Scenario: {scenario_name} ")
    # msg=0 silences the solver's internal log
    prob.solve(pulp.PULP_CBC_CMD(msg=0)) 

    # 8. Format and Return Results 
    if prob.status != pulp.LpStatusOptimal:
        print(f"Status: {pulp.LpStatus[prob.status]} (No optimal solution found for these constraints)")
        return None

    # Extract Solution 
    final_assignment = {}
    tail_durations = {t: 0 for t in tail_ids}
    
    for t in tail_ids:
        for f in flight_ids:
            if pulp.value(x[t][f]) == 1:
                final_assignment[f] = t
                tail_durations[t] += flight_times[f]

    # Calculate final scores
    final_cost = pulp.value(obj_cost)
    final_imbalance = pulp.value(obj_utilization_imbalance)
    final_buffer = pulp.value(obj_total_buffer)

    # Calculate variance (sum of squares from mean)
    durations = [d for d in tail_durations.values() if d > 0]
    mean_duration = sum(durations) / len(durations) if durations else 0
    score_util_variance = sum((d - mean_duration) ** 2 for d in tail_durations.values())
    
    result = {
        "feasible": True,
        "assignment": final_assignment,
        "scenario": scenario_name,
        "score_cost": round(final_cost, 2),
        "score_util_variance": round(score_util_variance, 2),
        "score_delay_buffer": round(final_buffer, 2), # Higher is better
        "score_util_imbalance": round(final_imbalance, 2), # Max - Min
        "tail_durations": tail_durations
    }
    
    print(f"Status: {pulp.LpStatus[prob.status]}")
    print(f"  Cost: {result['score_cost']}")
    print(f"  Imbalance (Max-Min): {result['score_util_imbalance']}")
    print(f"  Buffer: {result['score_delay_buffer']}")
    
    return result

# Main execution 
if __name__ == "__main__":
    
    print("Loading and pre-processing data from dataset.json...")
    problem_data = load_and_preprocess_data("dataset.json")
    
    if problem_data:
        all_results = []
        
        # Scenario 1: Pure Cost Focus
        # We don't set any limits, so the solver only minimizes cost.
        s1 = solve_tail_assignment_mip(
            problem_data, 
            scenario_name="Cost_Focus"
        )
        if s1: all_results.append(s1)

        # Scenario 2: Utilization Focus
        # We add a tight limit on imbalance (e.g., <= 90 minutes)
        s2 = solve_tail_assignment_mip(
            problem_data, 
            scenario_name="Util_Focus",
            util_imbalance_limit=90 
        )
        if s2: all_results.append(s2)

        # Scenario 3: Delay Focus
        # We demand a high amount of total buffer (e.g., >= 45 minutes)
        s3 = solve_tail_assignment_mip(
            problem_data, 
            scenario_name="Delay_Focus",
            min_buffer_required=45
        )
        if s3: all_results.append(s3)

        # Scenario 4: Balanced
        # We apply *both* constraints
        s4 = solve_tail_assignment_mip(
            problem_data, 
            scenario_name="Balanced",
            util_imbalance_limit=100,
            min_buffer_required=45
        )
        if s4: all_results.append(s4)

        print("\n Final Results (JSON) ")
        print(json.dumps(all_results, indent=4))
    else:
        print("Could not run solver due to data loading error.")

Loading and pre-processing data from dataset.json...

 Solving for Scenario: Cost_Focus 
Status: Optimal
  Cost: 545.0
  Imbalance (Max-Min): 315.0
  Buffer: 0.0

 Solving for Scenario: Util_Focus 
Status: Optimal
  Cost: 565.0
  Imbalance (Max-Min): 90.0
  Buffer: 0.0

 Solving for Scenario: Delay_Focus 
Status: Optimal
  Cost: 545.0
  Imbalance (Max-Min): 315.0
  Buffer: 75.0

 Solving for Scenario: Balanced 
Status: Infeasible (No optimal solution found for these constraints)

 Final Results (JSON) 
[
    {
        "feasible": true,
        "assignment": {
            "F1": "T1",
            "F3": "T1",
            "F4": "T1",
            "F2": "T2",
            "F5": "T2"
        },
        "scenario": "Cost_Focus",
        "score_cost": 545.0,
        "score_util_variance": 74418.75,
        "score_delay_buffer": 0.0,
        "score_util_imbalance": 315.0,
        "tail_durations": {
            "T1": 315,
            "T2": 210,
            "T3": 0
        }
    },
    {
        "

In [15]:
import pulp
import json
import math
from collections import defaultdict

#  1. DATA LOADER 

def load_and_preprocess_data(filename="dataset.json"):
    """
    Loads data and pre-calculates feasible connections and flight times.
    """
    try:
        with open(filename) as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Error: {filename} not found.")
        return None
    
    # Data Extraction
    flights_data = data['flights']
    tails_data = data['tails']
    costs = data['costs']

    # Data Structuring
    flight_ids = [f['flight_id'] for f in flights_data]
    tail_ids = [t['tail_id'] for t in tails_data]
    
    flight_times = {f['flight_id']: f['end_time'] - f['start_time'] for f in flights_data}
    flights_map = {f['flight_id']: f for f in flights_data}
    tails_map = {t['tail_id']: t for t in tails_data}

    # Pre-calculate Feasible Connections 
    feasible_connections = {}
    
    for t_id in tail_ids:
        min_turn = tails_map[t_id]['min_turnaround']
        for f_i in flights_data:
            for f_j in flights_data:
                if f_i['flight_id'] == f_j['flight_id']:
                    continue
                
                if f_i['dest'] == f_j['origin']:
                    gap = f_j['start_time'] - f_i['end_time']
                    if gap >= min_turn:
                        buffer = gap - min_turn
                        feasible_connections[(t_id, f_i['flight_id'], f_j['flight_id'])] = buffer
    
    return {
        "flight_ids": flight_ids,
        "tail_ids": tail_ids,
        "costs": costs,
        "flight_times": flight_times,
        "feasible_connections": feasible_connections,
        "flights_map": flights_map, # <-- Needed for fix
        "tails_map": tails_map      # <-- Needed for fix
    }

#  2. CONFLICT CHECKER 

def check_if_incompatible(f1, f2, tail_properties):
    """
    Checks if two flights conflict for a single tail.
    """
    min_turnaround = tail_properties['min_turnaround']
    
    # 1. Check for Time Overlap
    if max(f1['start_time'], f2['start_time']) < min(f1['end_time'], f2['end_time']):
        return True # Overlap
    
    # 2. Check f1 -> f2 sequential
    if f1['end_time'] <= f2['start_time']:
        if f1['dest'] != f2['origin']:
            return True # Location Mismatch
        if (f2['start_time'] - f1['end_time']) < min_turnaround:
            return True # Turnaround Violation
            
    # 3. Check f2 -> f1 sequential
    if f2['end_time'] <= f1['start_time']:
        if f2['dest'] != f1['origin']:
            return True # Location Mismatch
        if (f1['start_time'] - f2['end_time']) < min_turnaround:
            return True # Turnaround Violation
            
    return False # No conflict

# 3. FIXED SOLVER FUNCTION 

def solve_tail_assignment_mip(data, scenario_name, util_imbalance_limit=None, min_buffer_required=None):
    """
    Solves the full 3-objective MIP problem with the new incompatibility constraints.
    """
    
    # 1. Unpack Data 
    flight_ids = data['flight_ids']
    tail_ids = data['tail_ids']
    costs = data['costs']
    flight_times = data['flight_times']
    feasible_connections = data['feasible_connections']
    flights_map = data['flights_map'] 
    tails_map = data['tails_map']

    # 2. Model Initialization 
    prob = pulp.LpProblem(f"Tail_Assignment_{scenario_name}", pulp.LpMinimize)

    # 3. Decision Variables
    x = pulp.LpVariable.dicts("x", (tail_ids, flight_ids), 0, 1, pulp.LpBinary)
    y = pulp.LpVariable.dicts("y", feasible_connections.keys(), 0, 1, pulp.LpBinary)
    w_max = pulp.LpVariable("Workload_Max", 0)
    w_min = pulp.LpVariable("Workload_Min", 0)

    # 4. Define Objectives (as expressions) 
    obj_cost = pulp.lpSum(costs[t][f] * x[t][f] for t in tail_ids for f in flight_ids)
    workload = {t: pulp.lpSum(flight_times[f] * x[t][f] for f in flight_ids) for t in tail_ids}
    obj_utilization_imbalance = w_max - w_min
    obj_total_buffer = pulp.lpSum(b * y[(t, i, j)] for (t, i, j), b in feasible_connections.items())

    # 5. Set Main Objective
    prob += obj_cost, "Total_Cost_To_Minimize"

    # 6. Constraints

    # C1: Each flight is assigned exactly once
    for f in flight_ids:
        prob += pulp.lpSum(x[t][f] for t in tail_ids) == 1, f"C1_Assign_{f}"

    # C_NEW (THE FIX): Prevent Incompatible Assignments
    for t in tail_ids:
        tail_props = tails_map[t]
        for i in range(len(flight_ids)):
            for j in range(i + 1, len(flight_ids)):
                f_id_1 = flight_ids[i]
                f_id_2 = flight_ids[j]
                f_obj_1 = flights_map[f_id_1]
                f_obj_2 = flights_map[f_id_2]
                
                if check_if_incompatible(f_obj_1, f_obj_2, tail_props):
                    prob += x[t][f_id_1] + x[t][f_id_2] <= 1, f"C_Incompatible_{t}_{f_id_1}_{f_id_2}"

    # C2: Flow Conservation (Links x and y)
    for t in tail_ids:
        for f_i in flight_ids:
            flights_into_i = pulp.lpSum(y[(t, f_j, f_i)] for (tk, f_j, fik), _ in feasible_connections.items() if tk == t and fik == f_i)
            flights_out_of_i = pulp.lpSum(y[(t, f_i, f_j)] for (tk, fik, f_j), _ in feasible_connections.items() if tk == t and fik == f_i)
            prob += flights_into_i <= x[t][f_i], f"C2_In_{t}_{f_i}"
            prob += flights_out_of_i <= x[t][f_i], f"C3_Out_{t}_{f_i}"

    # C3: Utilization helper constraints
    for t in tail_ids:
        prob += w_max >= workload[t], f"C4_W_Max_{t}"
        prob += w_min <= workload[t], f"C4_W_Min_{t}" 

    # C4: Epsilon-Constraints
    if util_imbalance_limit is not None:
        prob += obj_utilization_imbalance <= util_imbalance_limit, "C5_Util_Limit"
    if min_buffer_required is not None:
        prob += obj_total_buffer >= min_buffer_required, "C6_Buffer_Limit"

    # 7. Solve 
    print(f"\n Solving for Scenario: {scenario_name} ") 
    prob.solve(pulp.PULP_CBC_CMD(msg=0)) 

    # 8. Format and Return Results 
    if prob.status != pulp.LpStatusOptimal:
        print(f"Status: {pulp.LpStatus[prob.status]} (No optimal solution found for these constraints)")
        return None

    final_assignment = {}
    tail_durations = {t: 0 for t in tail_ids}
    for t in tail_ids:
        for f in flight_ids:
            if pulp.value(x[t][f]) == 1:
                final_assignment[f] = t
                tail_durations[t] += flight_times[f]

    final_cost = pulp.value(obj_cost)
    final_buffer = pulp.value(obj_total_buffer)
    used_durations = [d for d in tail_durations.values() if d > 0]
    final_imbalance = max(used_durations) - min(used_durations) if used_durations else 0
    mean_duration = sum(flight_times.values()) / len(tail_ids)
    score_util_variance = sum((d - mean_duration) ** 2 for d in tail_durations.values())
    
    result = {
        "feasible": True, "assignment": final_assignment, "scenario": scenario_name,
        "score_cost": round(final_cost, 2), "score_util_variance": round(score_util_variance, 2),
        "score_delay_buffer": round(final_buffer, 2), "score_util_imbalance": round(final_imbalance, 2),
        "tail_durations": tail_durations
    }
    
    print(f"Status: {pulp.LpStatus[prob.status]}")
    print(f"  Cost: {result['score_cost']}")
    print(f"  Imbalance (Max-Min): {result['score_util_imbalance']}")
    print(f"  Buffer: {result['score_delay_buffer']}")
    
    return result

# 4. VALIDATION FUNCTIONS 

def load_validation_data(filename="dataset.json"):
    """ Loads data into maps for easy validation lookups. """
    with open(filename) as f:
        data = json.load(f)
    flights_map = {}
    for f in data['flights']:
        f['duration'] = f['end_time'] - f['start_time']
        flights_map[f['flight_id']] = f
    tails_map = {t['tail_id']: t for t in data['tails']}
    return {
        "flights_map": flights_map, "tails_map": tails_map,
        "costs": data['costs'], "flight_ids": list(flights_map.keys()),
        "tail_ids": list(tails_map.keys())
    }

def validate_solution(assignment, all_data):
    """ Checks a given assignment for feasibility and calculates its scores. """
    print("---" * 10)
    print(f"Validating assignment: {assignment}")
    is_feasible = True
    errors = []
    
    if assignment is None:
        print("❌ SOLUTION IS INFEASIBLE (Solver found no solution)")
        return

    flights_map = all_data['flights_map']
    tails_map = all_data['tails_map']
    costs = all_data['costs']

    # Check 1: Flight Coverage 
    assigned_flights = set(assignment.keys())
    required_flights = set(all_data['flight_ids'])
    if assigned_flights != required_flights:
        is_feasible = False
        missing = required_flights - assigned_flights
        if missing: errors.append(f"Missing flight assignments: {missing}")

    # Check 2: Conflicts
    tail_schedule = defaultdict(list)
    for flight_id, tail_id in assignment.items():
        if flight_id in flights_map:
            tail_schedule[tail_id].append(flights_map[flight_id])

    for tail_id, assigned_flights in tail_schedule.items():
        min_turnaround = tails_map[tail_id]['min_turnaround']
        for i in range(len(assigned_flights)):
            for j in range(i + 1, len(assigned_flights)):
                f1, f2 = assigned_flights[i], assigned_flights[j]
                conflict_found, reason = False, ""

                if max(f1['start_time'], f2['start_time']) < min(f1['end_time'], f2['end_time']):
                    conflict_found, reason = True, f"Time Overlap: {f1['flight_id']} (...) and {f2['flight_id']} (...)"
                elif f1['end_time'] <= f2['start_time']:
                    if f1['dest'] != f2['origin']:
                        conflict_found, reason = True, f"Location Mismatch: {f1['flight_id']} lands {f1['dest']}, {f2['flight_id']} departs {f2['origin']}"
                    elif (f2['start_time'] - f1['end_time']) < min_turnaround:
                        gap = f2['start_time'] - f1['end_time']
                        conflict_found, reason = True, f"Turnaround Violation ({gap}m < {min_turnaround}m)"
                elif f2['end_time'] <= f1['start_time']:
                    if f2['dest'] != f1['origin']:
                        conflict_found, reason = True, f"Location Mismatch: {f2['flight_id']} lands {f2['dest']}, {f1['flight_id']} departs {f1['origin']}"
                    elif (f1['start_time'] - f2['end_time']) < min_turnaround:
                        gap = f1['start_time'] - f2['end_time']
                        conflict_found, reason = True, f"Turnaround Violation ({gap}m < {min_turnaround}m)"

                if conflict_found:
                    is_feasible = False
                    errors.append(f"Tail {tail_id}: {reason}")

    # 4. Final Verdict and Scoring 
    if not is_feasible:
        print("\n❌ SOLUTION IS INVALID")
        for err in errors: print(f"   - {err}")
    else:
        print("\n✅ SOLUTION IS VALID (FEASIBLE)")
        # (Scores are already printed by the solver function, so we don't need to repeat)

# 5. MAIN EXECUTION BLOCK 

print("Loading data for solver and validator...")
problem_data = load_and_preprocess_data("dataset.json")
validation_data = load_validation_data("dataset.json")

if problem_data:
    print("\n====== VALIDATING PULP SCENARIOS ======")
    
    # Scenario 1: Cost_Focus 
    s1 = solve_tail_assignment_mip(problem_data, "Cost_Focus")
    validate_solution(s1['assignment'] if s1 else None, validation_data)

    # Scenario 2: Util_Focus
    s2 = solve_tail_assignment_mip(problem_data, "Util_Focus", util_imbalance_limit=90)
    validate_solution(s2['assignment'] if s2 else None, validation_data)

    # Scenario 3: Delay_Focus
    s3 = solve_tail_assignment_mip(problem_data, "Delay_Focus", min_buffer_required=45)
    validate_solution(s3['assignment'] if s3 else None, validation_data)

    # Scenario 4: Balanced
    s4 = solve_tail_assignment_mip(problem_data, "Balanced", util_imbalance_limit=100, min_buffer_required=45)
    validate_solution(s4['assignment'] if s4 else None, validation_data)
else:
    print("Could not run solver due to data loading error.")

Loading data for solver and validator...

====== VALIDATING PULP SCENARIOS ======

 Solving for Scenario: Cost_Focus 
Status: Optimal
  Cost: 585.0
  Imbalance (Max-Min): 105
  Buffer: 0.0
------------------------------
Validating assignment: {'F1': 'T1', 'F5': 'T1', 'F2': 'T2', 'F3': 'T2', 'F4': 'T3'}

✅ SOLUTION IS VALID (FEASIBLE)

 Solving for Scenario: Util_Focus 
Status: Infeasible (No optimal solution found for these constraints)
------------------------------
Validating assignment: None
❌ SOLUTION IS INFEASIBLE (Solver found no solution)

 Solving for Scenario: Delay_Focus 
Status: Optimal
  Cost: 585.0
  Imbalance (Max-Min): 105
  Buffer: 45.0
------------------------------
Validating assignment: {'F1': 'T1', 'F5': 'T1', 'F2': 'T2', 'F3': 'T2', 'F4': 'T3'}

✅ SOLUTION IS VALID (FEASIBLE)

 Solving for Scenario: Balanced 
Status: Infeasible (No optimal solution found for these constraints)
------------------------------
Validating assignment: None
❌ SOLUTION IS INFEASIBLE (Solv